In [1]:
from openpyxl import *
from pandas import *

In [2]:
prices = [] #Тут будут храниться цена каждого продукта (понадобится для сегментации)
products_info = DataFrame({'Price_sum': [], 'Quantity': []}) #Таблица, где хранятся все данные

In [3]:
def add_data(file): #Добавляет данные в таблицу (Берет данные из файла)
    global products_info, prices
    for i in range(3, file.max_row + 1):
        brand = file.cell(row = i, column = 2).value
        volume = float(file.cell(row = i, column = 3).value.split()[0].replace(',', '.'))
        price = file.cell(row = i, column = 4).value / volume
        prices.append(price)
        if not brand in products_info.index:
            products_info = DataFrame([[price, 1]], columns = ['Price_sum', 'Quantity'], index=[brand]).append(products_info)
        else:
            products_info.at[brand, 'Price_sum'] += price
            products_info.at[brand, 'Quantity'] = products_info.loc[brand]['Quantity'] + 1
            
def create(): #Создает отчет
    global products_info
    wb = Workbook()
    ws = wb.active
    ws.append(['Бренд', 'Ценовой сегмент'])
    for brand in products_info.index:
        ws.append([brand, products_info.loc[brand, 'Segment']])
    wb.save('Сегментация брендов.xlsx')



In [4]:
#Начало основной программы
file_name = 'products.xlsx'
file_time = load_workbook(file_name)
for name in file_time.get_sheet_names():
    file = file_time.get_sheet_by_name(name) #В этой переменной хранится одна страничка из Эксель-файла
    add_data(file)

prices = sorted(prices) #Сортирует массив с ценами в порядке возрастания
l_m_id = len(prices) // 4 #Индекс цены, которая разделяет бренды Low и Middle
m_u_id = len(prices) // 2 #Индекс цены, которая разделяет бренды Middle и Upper Middle
u_h_id = l_m_id * 3 #Индекс цены, которая разделяет бренды Upper Middle и High

#Методология сегментации заключается в следующем: У нас есть в массиве все данные о ценах.
#Первая четверть цен относится к ценовой области Low Priced
#Вторая - к Middle Priced
#И так далее

products_info['Average_price'] = round(products_info['Price_sum'] / products_info['Quantity']) #Добавляет столбец средней цены

for brand in products_info.index: #Добавляет столбец сегментации
    if products_info.loc[brand]['Average_price'] <= prices[l_m_id]:
        products_info.at[brand, 'Segment'] = 'Low Priced'
    elif products_info.loc[brand]['Average_price'] <= prices[m_u_id]:
        products_info.at[brand, 'Segment'] = 'Middle Priced'
    elif products_info.loc[brand]['Average_price'] <= prices[u_h_id]:
        products_info.at[brand, 'Segment'] = 'Upper Middle Priced'
    else:
        products_info.at[brand, 'Segment'] = 'High Priced'

print(products_info)
answer = input('Сохранить отчет в эксель-файл? Напишите, Да или Нет: ')
if answer == 'Да':
    create()

C:\Program Files\Anaconda\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated function get_sheet_names (Use wb.sheetnames).
  after removing the cwd from sys.path.
C:\Program Files\Anaconda\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
  """


                   Price_sum  Quantity  Average_price              Segment
Valio             943.900000       6.0          157.0          High Priced
Простоквашино     272.315789       3.0           91.0  Upper Middle Priced
Parmalat          875.780000       3.0          292.0          High Priced
Дмитровский       219.980000       3.0           73.0           Low Priced
Домик в деревне  1343.960171      15.0           90.0        Middle Priced


Сохранить отчет в эксель-файл? Напишите, Да или Нет:  Нет
